# 2022, day 17

Part 2 requires finding a pattern in the stack heights to identify at what frequency the pattern repeats and then estimate the stack hight after 1_000_000_000_000 rocks have dropped.

Load the file with stack heights after each rock

In [1]:
import pandas as pd

In [2]:
!ls -alF *.csv

-rw-r--r-- 1 bjoern bjoern 111570 Jan  2 19:01 202217_2.csv
-rw-r--r-- 1 bjoern bjoern 111895 Jan  2 19:24 202217_2_riddle.csv


In [3]:
df = pd.read_csv('202217_2.csv', names=['rock', 'height'])

In [4]:
df.head()

,rock,height
0,0,-1
1,1,-4
2,2,-6
3,3,-7
4,4,-9


## Approach

- pick a starting point, e.g. rock 1000
- check height differences for 3 cycles of length n and see if they have the same height difference. Start with a reasonable n, e.g. 30
- once the length of the cycle is found, we need to find the starting point. Do a binary search down from starting point and check if the same height difference from n is found

Test how we can generate a series of height differences from starting point at multiples of n. Then check that all elements in the series are the same.

In [5]:
n = 30
starting_point = 1000
indices = [starting_point + n * i for i in range(4)]
df_diff = df.iloc[indices]['height'].diff().iloc[1:]
(df_diff == df_diff.iloc[0]).all()

False

In [6]:
starting_point = 1000
n = 30
while True:
    indices = [starting_point + n * i for i in range(4)]
    df_diff = df.iloc[indices]['height'].diff().iloc[1:]
    if (df_diff == df_diff.iloc[0]).all():
        break
        
    n += 1
    

cycles = n
height = int(-df_diff.iloc[0])

print(f'Found cycle length: {cycles}, height is {height}')       

Found cycle length: 35, height is 53


Binary search for the correct starting point, from 1000 downwards

In [7]:
low = 0
high = 1000
while (high - low) > 1:
    mid = low + (high - low) // 2
    print(f'Search in {low=} {high=}, {mid=}')
    indices = [mid + cycles * i for i in range(4)]
    df_diff = df.iloc[indices]['height'].diff().iloc[1:]
    if (df_diff == df_diff.iloc[0]).all():
        # too high, go with lower half interval
        high = mid
    else:
        low = mid
        
print(f'{low=} {mid=} {high=}')

Search in low=0 high=1000, mid=500
Search in low=0 high=500, mid=250
Search in low=0 high=250, mid=125
Search in low=0 high=125, mid=62
Search in low=0 high=62, mid=31
Search in low=0 high=31, mid=15
Search in low=0 high=15, mid=7
Search in low=7 high=15, mid=11
Search in low=11 high=15, mid=13
Search in low=13 high=15, mid=14
low=13 mid=14 high=14


Found the results, now calculate the solution, which is:

height of first 413 
+ (1_000_000_000_000 - 413) // cycles * height
+ height of (1_000_000_000_000 - 413) % cycles of the sequence

Result: 1580758017509

Test case: 1514285714288

In [8]:
low_part = -df.iloc[low]['height']
print(low_part)
mid_part = ((1_000_000_000_000 - mid) // cycles) * height
print(mid_part)
remaining_rocks = ((1_000_000_000_000 - mid) % cycles)
print(remaining_rocks)
upper_part = -df.iloc[remaining_rocks + low]['height'] + df.iloc[low]['height']
print(upper_part)
print('Result: ', low_part + mid_part + upper_part)

23
1514285714263
1
2
Result:  1514285714288
